In [ ]:
!pip install torch
!pip install opencv-python
!pip install pandas
!pip install ultralytics

In [ ]:
import cv2
import torch
import pandas as pd
from google.colab.patches import cv2_imshow
import datetime

# Define o caminho para o arquivo de vídeo
# Certifique-se de que o nome e a pasta estejam corretos no seu ambiente do Colab
video_path = "/content/video_sprint3.mp4"

# Lista para armazenar o log de detecções
# Cada entrada na lista será um dicionário com os dados de um frame
detections_log = []

# Define o tempo de início e fim para o processamento do vídeo, em segundos
start_time_sec = 6
end_time_sec = 9

# Carrega o modelo YOLOv5 pré-treinado da biblioteca 'ultralytics'
# O 'yolov5s' é a versão menor e mais rápida do modelo
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
# Configura o modelo para detectar apenas a classe 'moto', que tem o ID 3
model.classes = [3]

# Inicializa a captura de vídeo a partir do arquivo
cap = cv2.VideoCapture(video_path)

# Verifica se o arquivo de vídeo foi aberto corretamente
if not cap.isOpened():
    print(f"Erro: Não foi possível abrir o arquivo de vídeo em {video_path}")
else:
    # --- Configuração do Ponto de Partida e Fim do Vídeo ---
    # Converte o tempo de início de segundos para milissegundos
    start_time_ms = start_time_sec * 1000
    # Define a posição inicial do vídeo
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time_ms)

    print(f"Processando o vídeo do segundo {start_time_sec} ao {end_time_sec}...")

    # Converte o tempo de fim de segundos para milissegundos
    end_time_ms = end_time_sec * 1000

    try:
        # Loop principal para processar o vídeo frame por frame
        while True:
            # Lê um novo frame do vídeo e uma variável de retorno (ret)
            ret, frame = cap.read()

            # Condição para parar o loop:
            # 1. Se 'ret' for False (vídeo terminou ou erro na leitura)
            # 2. Se a posição atual do vídeo for maior ou igual ao tempo de fim
            if not ret or cap.get(cv2.CAP_PROP_POS_MSEC) >= end_time_ms:
                print(f"Processamento parado. O tempo limite de {end_time_sec} segundos foi atingido ou o vídeo terminou.")
                break

            # Realiza a detecção de objetos no frame atual
            results = model(frame)
            # Extrai os resultados das detecções para um DataFrame do Pandas
            detections = results.pandas().xyxy[0]

            # --- Lógica para o Log e Persistência de Dados ---
            # Conta o número de motos detectadas no frame
            num_motos = len(detections)
            # Calcula a confiança média das detecções, se houver motos
            if num_motos > 0:
                avg_confidence = detections['confidence'].mean()
            else:
                avg_confidence = 0

            # Cria uma entrada de log com timestamp, contagem e confiança
            log_entry = {
                'timestamp': datetime.datetime.now().isoformat(),
                'num_motos_detectadas': num_motos,
                'confianca_media': avg_confidence
            }
            # Adiciona a entrada à lista de log
            detections_log.append(log_entry)

            # --- Lógica para o Output Visual ---
            # Loop para desenhar as caixas delimitadoras para cada moto detectada
            for _, detection in detections.iterrows():
                # Extrai as coordenadas da caixa e a confiança
                xmin, ymin, xmax, ymax = int(detection['xmin']), int(detection['ymin']), int(detection['xmax']), int(detection['ymax'])
                confidence = detection['confidence']

                # Desenha o retângulo verde ao redor da moto
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                # Cria o texto com a pontuação de confiança
                text = f"Moto: {confidence:.2f}"
                # Adiciona o texto acima da caixa
                cv2.putText(frame, text, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Exibe o frame processado no ambiente do Google Colab
            cv2_imshow(frame)

    except Exception as e:
        print(f"Ocorreu um erro: {e}")

# Libera o objeto de captura de vídeo
cap.release()

# --- Salva o Log de Detecções em um Arquivo CSV ---
# Verifica se a lista de log não está vazia antes de salvar
if detections_log:
    df_log = pd.DataFrame(detections_log)
    df_log.to_csv('historico_deteccoes.csv', index=False)
    print("\nHistórico de detecções salvo como 'historico_deteccoes.csv'")

print("\nProcessamento concluído.")

In [ ]:
df_log = pd.read_csv('historico_deteccoes.csv')

print(df_log)